In [ ]:
from specsscan.core import SpecsScan
import matplotlib.pyplot as plt

In [ ]:
sps = SpecsScan()

In [ ]:
print(sps)

In [ ]:
print(sps.spa)

In [ ]:
from specsscan.settings import parse_config

In [ ]:
parse_config({'spa_params' : {'fft_filter_peaks': []}})

In [ ]:
print(parse_config("../tests/data/config.yaml"))

In [ ]:
config = parse_config("../tests/data/config.yaml")

In [ ]:
import json
with open("../tests/data/config.json", "w") as outfile:
    json.dump(config, outfile)

In [ ]:
from specsscan.settings import load_config
from pathlib import Path

In [ ]:
with open("../tests/data/config.json") as file:
    json.load(file)